# Clustering

In [1]:
import ais_processing
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import MultiLineString, Polygon, Point

In [7]:
df = gpd.read_file('bay_transits.csv', GEOM_POSSIBLE_NAMES='geometry', KEEP_GEOM_COLUMNS='NO')

In [8]:
df.shape

(14506, 10)

In [11]:
df = df.rename(columns={'max_SOG': 'Speed', 'LOA': 'Length'})
df.head()

,start_DTG,end_DTG,SOG,LOA_unbinned,day,month,length,Speed,Length,geometry
0,2015-01-19 18:21:43,2015-01-20 00:03:48,47.7,28.65,Mon,Jan,7.900541642041586,45-50kt,0-50m,"LINESTRING (-122.38206 37.74708, -122.38202 37..."
1,2015-01-11 18:39:55,2015-01-12 04:13:36,11.3,27.42,Sun,Jan,1.5502465242070502,10-15kt,0-50m,"LINESTRING (-121.90111 38.04800, -121.90470 38..."
2,2015-01-16 09:12:44,2015-01-16 20:13:05,13.0,27.42,Fri,Jan,1.5471122663391763,10-15kt,0-50m,"LINESTRING (-121.90003 38.04878, -121.90000 38..."
3,2015-01-09 05:25:13,2015-01-09 15:02:24,9.9,27.42,Fri,Jan,1.5416339346080299,5-10kt,0-50m,"LINESTRING (-121.90071 38.04712, -121.90374 38..."
4,2015-01-10 05:47:36,2015-01-10 15:15:06,10.9,27.42,Sat,Jan,1.5387565264193501,10-15kt,0-50m,"LINESTRING (-121.90134 38.04802, -121.90414 38..."


In [12]:
clusters = ais_processing.cluster_transits(df)

In [13]:
foo = ais_processing.cluster_df(df, clusters)

In [14]:
foo.to_csv('transits_with_routes.csv', index=False)

In [16]:
foo.Route

0          1
1          1
2          1
3          1
4          1
        ... 
3785    1133
3786    1133
3787    1133
3788    1133
3789    1133
Name: Route, Length: 3790, dtype: int64

In [ ]:
n_clusters = np.array([len(cluster) for cluster in clusters])
sig_clusters = np.where(n_clusters >= 10)[0]

day = []
Speed = []
Length = []
Count = []
geometry = []

# TODO: replace below with a loop
cluster = clusters[sig_clusters[16]]

day_speed_len = set([xyz for xyz in zip(df.day[cluster], df.Speed[cluster], df.Length[cluster])])
for dsl in day_speed_len:
    day.append(dsl[0])
    Speed.append(dsl[1])
    Length.append(dsl[2])
    sub_cluster = np.array(cluster)[np.where((df.day[cluster] == dsl[0]) & (df.Speed[cluster] == dsl[1]) & (df.Length[cluster] == dsl[2]))[0]]
    n_lines = np.ceil(len(sub_cluster) / 10)
    Count.append(len(sub_cluster) / n_lines)
    geoms = np.array(df.geometry[sub_cluster])
    geom_index = np.random.choice(sub_cluster, np.int(n_lines), replace=False)
    lines = MultiLineString(list(df.geometry[geom_index]))
    geometry.append(lines)

In [ ]:
Count

In [ ]:
geometry[10]

In [ ]:
n_lines

In [ ]:
for i in day_speed_len:
    day.append(i[0])
    Speed.append(i[1])
    Length.append(i[2])
    indicies = 
    geometries = 
    print(np.sum((df.day[cluster] == i[0]) & (df.Speed[cluster] == i[1]) & (df.Length[cluster] == i[2])))

In [ ]:
days = df.day[cluster].unique()
speeds = df.Speed[cluster].unique()
lengths = df.Length[cluster].unique()
#permutations = np.array(np.meshgrid(days, speeds, lengths)).T.reshape(-1,3)

#### Write to a new gpd.GeoDataFrame broken down by characteristics

In [ ]:
days = list(df.day.unique())
speeds = list(df.Speed.unique())
lengths = list(df.Length.unique())
permutations = np.array(np.meshgrid(days, speeds, lengths)).T.reshape(-1,3)

In [ ]:
permutations.shape

In [ ]:
dfi = df[(df.day == 'Mon') & (df.Speed == '0-10kt') & (df.Length == '0-100m')].reset_index(drop=True)
clusters = ais_processing.cluster_transits(dfi)

In [ ]:
n_clusters = np.array([len(cluster) for cluster in clusters])

In [ ]:
n_clusters.max()

In [ ]:
if n_clusters.max() > 10:
    

In [ ]:
[np.random.choice(x, p=[0.1]) for x in clusters]

In [ ]:
Day = permutations[:,0]
Speed = permutations[:,1]
Length = permutations[:,2]
geometry = []

pd.DataFrame.from_dict({'Day': Day, 'Speed': Speed, 'Length': Length})

In [ ]:
permutations[:,0]

#### Build a test *non-production* gpd.GeoDataFrame that captures the clustered routes to test performance on the site

In [ ]:
max_SOG  = [df.max_SOG[transit].reset_index(drop=True)[0] for transit in transits]
LOA      = [df.LOA[transit].reset_index(drop=True)[0] for transit in transits]
geometry = [df.geometry[transit].reset_index(drop=True)[0] for transit in transits]

In [ ]:
pd.DataFrame.from_dict({'max_SOG': max_SOG, 'LOA': LOA, 'geometry': geometry})

In [ ]:
vessels = gpd.GeoDataFrame.from_dict({'Speed': max_SOG, 'Length': LOA, 'Geometry': geometry})

In [ ]:
vessels.to_csv('bay_traffic.csv')